In [ ]:
import re
import requests
import time
import bs4
import json
import pandas as pd
import jieba
import jieba.analyse
import gensim
import emoticons

In [ ]:
BilibiliWords = pd.read_csv('BilibiliWords.txt', \
                                header = None, delimiter="\t", quoting=3, error_bad_lines=False).loc[:,0].tolist()

jieba.load_userdict(BilibiliWords)
SChineseStopWords = pd.read_csv('ChineseStopwords.txt',
                                     header = None, delimiter="\t", quoting=3, error_bad_lines=False).loc[:,0].tolist()


SChineseStopWords += [' ',',']

In [ ]:
def getBLVData(video_url):
    cid = None
    r = requests.get(video_url)
    if r is not None:
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        
        #scrape A version
        title = soup.find('div', class_ = 'v-title')
        if title is not None:
            vname = title.text
            #video category
            category = ""
            aTags = soup.findAll('a', href=re.compile('/video'))
            for aTag in aTags:
                category = category + "/" + aTag.text
                #unique ids
                idstr = soup.find('div', class_ ='scontent').text
                if re.search('cid=\d+',idstr) is not None:
                    cid = re.search('cid=\d+',idstr).group()[4:]
                    aid = re.search('aid=\d+',idstr).group()[4:]
        else:
            TONGSHAWOYE1 = 'window.__INITIAL_STATE__='
            TONGSHAWOYE2 = ';(function(){var s;(s=document.currentScript||document.scripts[document.scripts.length-1]).parentNode.removeChild(s);}());'
    
            jsonTags = soup.findAll('script',type = False)
            for jsonTag in jsonTags:
                if jsonTag.text.startswith(TONGSHAWOYE1) and jsonTag.text.endswith(TONGSHAWOYE2):
                    data = json.loads(jsonTag.text[25:-122])
                    if 'aid' in data and 'videoData' in data:
                        aid = data['aid']
                        vname = str(data['videoData']['title'])
                        cid = str(data['videoData']['pages'][0]['cid'])
                        main_cat = "/"+data['videoData']['breadcrumb']['first']['name']
                        sec_cat = "/"+data['videoData']['breadcrumb']['second']['name']
                        category = main_cat + sec_cat + "/高级弹幕"
        
        if cid is not None:
            bsxml = 'http://comment.bilibili.tv/'+ cid+'.xml'
            bsr = requests.get(bsxml)
            if bsr is not None:
                bs_list = []
                bssoup = bs4.BeautifulSoup(bsr.text, 'xml')
                dTags = bssoup.findAll('d', p = True)
                for dTag in dTags:
                    bs_list.append(dTag.text)
                return (vname, category, aid, cid, bs_list)

In [ ]:
def get_rank(filename):
    '''
    Get a list of videos in the rank.
    Input: filename - the name of the json file as a string
    Example:
        'all-3-0.json'
    Return:
        a list of aid of videos in the ranking
    '''
    aid_lst = []
    rank = json.load(open(filename))
    v_lst = rank['rank']['list']
    for item in v_lst:
        aid_lst.append(int(item['aid']))
        if 'others' in item:
            for other in item['others']:
                aid_lst.append(int(other['aid']))
    return aid_lst

In [ ]:
def cleanBStext(BStext):
    loc = BStext.find('var names =')
    if loc:
        BStext = BStext[:loc]
    return BStext

In [ ]:
def get_df(filename):
    '''
    Get the desired dataframe from the imported json file.
    Example:
        'all-3-0.json'
    Return:
        The desired dataframe from this json file.
    '''
    l = get_rank(filename)
    parsDict = {'video_title':[], 'video_url':[], 'category':[], 'BS_text':[] }
    for n in l:
        time.sleep(2)
        video_url = 'https://www.bilibili.com/video/av{}/'.format(n)
        if getBLVData(video_url):
            result = getBLVData(video_url)
            parsDict['video_title'].append(result[0])
            parsDict['video_url'].append(video_url)
            parsDict['category'].append(result[1])
            parsDict['BS_text'].append(result[4])
    BLDF = pd.DataFrame(parsDict)
    BLDF['main category'] = BLDF['category'].str.split('/').map(lambda x: x[1])
    BLDF['sub category'] = BLDF['category'].str.split('/').map(lambda x: x[2])
    BLDF['BS_text'] = BLDF['BS_text'].apply(lambda x: ''.join(x)).apply(lambda x: cleanBStext(x))
    return BLDF

In [ ]:
def Remove_SChinese_Stopwords(words, list_of_stopwords):
    output = [word for word in words if word not in list_of_stopwords]
    return output

In [ ]:
EMOTICONS = emoticons.EMOTICONS

def str_find_all(emot, bulletstr, start = 0, sub = None, lstps = None): 
    if lstps == None: 
        lstps = []
    if sub == None: 
        sub = bulletstr
    if emot not in sub: 
        return lstps
    elif start >= len(bulletstr) - 1: 
        return lstps

    else: 
        subpos = sub.find(emot)
        pos = start + subpos
        lstps.append(pos)
        start = pos + 1
        sub = bulletstr[start:]
        lst_sub = str_find_all(emot, bulletstr, start, sub, lstps)

        return lstps



def jpemoticon(bulletstr): 

    dict_pos = {}

    for emotlst in EMOTICONS: 
        for icon in EMOTICONS[emotlst]: 
            if icon in bulletstr: 
                positions = str_find_all(icon, bulletstr)
                if icon in dict_pos: 
                    dict_pos[icon].extend(positions)
                else: 
                    dict_pos[icon] = positions

    return dict_pos

In [ ]:
def emot_find_all(emot, bulletstr, start = 0, sub = None, lstps = None): 
    if lstps == None: 
        lstps = []
    if sub == None: 
        sub = bulletstr
    if emot not in sub: 
        return lstps
    elif start >= len(bulletstr) - 1: 
        return lstps

    else: 
        subpos = sub.find(emot)
        pos = start + subpos
        end = pos + len(emot)
        lstps.append((pos,end))
        start = pos + 1
        sub = bulletstr[start:]
        lst_sub = emot_find_all(emot, bulletstr, start, sub, lstps)

        return lstps

In [ ]:
AllEmoticons = sum(EMOTICONS.values(),[])

In [ ]:
Add_Emoticons = ['(╯‵□′)╯︵┻━┻',
                 '( ゜-  ゜)つロ',
                 '（￣▽￣）',
                 '(• ̀ㅁ• ́;)',
                 'ヾ(@^▽^@)ノ',
                 '\(≧▽≦)/',
                 '(＾Ｕ＾)ノ',
                 '( ﾟ ∀ ﾟ )',
                 '(๑• ̀ㅂ• ́)و✧',
                 '(´இ皿இ｀)',
                 '(´◑д◐｀)',
                 'q(^_^q=)>',
                 '<(=p^_^)p',
                 '(・^・╬)',
                 '(ノ=Д=)ノ┻━┻',
                 '(# ﾟД ﾟ)',
                 '( ⊙  o  ⊙ )',
                 '(ˉ﹃ˉ)',
                 '_(:3」∠)_',
                 '_(:з」∠)_',
                 '(ﾉ´∀｀*)ノ',
                 '(;゜ロ゜ノ)ノ',
                 '(σ｀д′)σ',
                 '(￣_,￣ )',
                 '(￣ε(#￣)',
                 '☆╰╮(￣▽￣///)',
                 '░▒▓█▇▅▂∩(･ω･)∩▂▅▇█▓▒░',
                 'o(╯□╰)o',
                 '(ಥ_ಥ)',
                 '(╯°Д°)╯︵┴┴',
                 '*^_^*',
                 '(;´༎ ຶД༎ ຶ`)',
                 '←_←',
                 '(๑• . •๑)',
                 '*罒▽罒*',
                 '(⑉°з°)-♡',
                 'o(╯□╰)o',
                 '(￢∀￢)σ',
                 '＜（－︿－）＞',
                 '( =①ω①=)',
                 '( ⊙ o ⊙ )',
                 '(ｏ  ‵-′)ノ”(ノ﹏<。)',
                 '٩(*∂v∂*)۶',
                 ' _(• ̀ω• ́」∠)_',
                 'Σ（・□・；）',
                 '╭(╯^╰)╮',
                 '╭(￣▽￣)╮',
                 'Σ(￣□￣||)',
                 '(<ゝω·)☆',
                 ' (๑• ̀ㅂ• ́)و✧',
                 '(●´∇｀●)',
                 '_:(´ཀ`」∠):_',
                 '(๑‾ ᷆д‾ ᷇๑)',
                 '( ๑ŏ  ﹏  ŏ๑ )',
                 '(°ー°〃)',
                 '(*+﹏+*)',
                 '(^・ω・^  )',
                 '～(￣▽￣～)',
                 '(～￣▽￣)～',
                 '(*^__^*)',
                 '(^・ω・^  )',
                 ' o(≧口≦)o']

AllEmoticons += Add_Emoticons

In [ ]:
def smart_cut(string, patterns, user_dict = None):
    output = []
    str_start = 0
    str_end = 0
    
    if isinstance(string, str):
        all_positions = sum([emot_find_all(emot, string) for emot in AllEmoticons],[])
        all_positions.sort()
        
        if user_dict is not None:
            jieba.load_userdict(user_dict)
            
        if all_positions == []:
            output = jieba.lcut(string)
        else:
            for pos in all_positions:
                str_end = pos[0]
                wordslist = jieba.lcut(string)[str_start:str_end]
                output += wordslist
                output.append(string[pos[0]:pos[1]])
                str_start = pos[1]
    
    return output

In [ ]:
def final_df(filename):
    BLDF = get_df(filename)
    BLDF['segmented'] = BLDF['BS_text'].astype(str).apply(jieba.lcut)
    BLDF['clean_text'] = BLDF['segmented'].apply(Remove_SChinese_Stopwords,
                                             list_of_stopwords = SChineseStopWords)
    BLDF['Smart_Cutted'] = BLDF['BS_text'].apply(smart_cut, patterns = AllEmoticons, user_dict = BilibiliWords)
    BLDF['normalized_words'] = BLDF['Smart_Cutted'].apply(Remove_SChinese_Stopwords,
                                                      list_of_stopwords = SChineseStopWords)
    taggedDocs = []
    for index, row in BLDF.iterrows():
        #Just doing a simple keyword assignment
        docKeywords = []#s for s in keywords if s in row['normalized_words']
        docKeywords.append(row['video_title'])
        docKeywords.append(row['category'])
        taggedDocs.append(gensim.models.doc2vec.TaggedDocument(words = row['normalized_words'], tags = docKeywords))
    BLDF['TaggedText'] = taggedDocs
    return BLDF

In [ ]:
filename_list = ['动画all-30-1.json', '国创相关all-30-168.json', '娱乐all-30-5.json', '影视all-30-181.json', \
                '时尚all-30-155.json', '游戏all-30-4.json', '生活all-30-160.json', '科技all-30-36.json', \
                '舞蹈all-30-129.json', '音乐all-30-3.json', '鬼畜all-30-119.json']
for filename in filename_list:
    BLDF = final_df(filename)
    BLDF.to_csv('/Users/apple/Desktop/BLData{}.txt'.format(filename[0: 2]), header = True, index = None, sep = "|", mode = 'a')